In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import datetime

def create_tensorboard_callback(folder_name):
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + "-" + folder_name
    return tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


In [0]:
import sys
print(sys.version)

3.6.9 (default, Apr 18 2020, 01:56:04) 
[GCC 8.4.0]


In [0]:
def starting_model(num_classes):
    return tf.keras.models.Sequential(layers=[
        tf.keras.layers.Conv2D(10, (3,3), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.Conv2D(20, (3,3), activation='relu'),  
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(40, (3,3), activation='relu'),   
        tf.keras.layers.Conv2D(80, (3,3), activation='relu'),   
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(160, (3,3), activation='relu'), 
        tf.keras.layers.Conv2D(320, (3,3), activation='relu'),  
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]), 'starting', '--'

In [0]:
from keras.callbacks import ReduceLROnPlateau

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.0001)

In [0]:
def prepare(df, is_training=True):
    X = None
    y = None

    if is_training:
      X = df.iloc[:,1:]
      y = df.iloc[:,0]
    else:
      X = df

    # normalizing input from [0,255] to [0,1]
    X = X / 255.0

    # each row is image of 28x28, one channel.
    X = np.reshape(X.values, (X.shape[0], 28, 28, 1))
    if is_training:
      y = y.values

      num_classes = 10
      y = tf.keras.utils.to_categorical(y, num_classes=num_classes)
    
    return X,y

In [0]:
batch_size = 128
epochs = 30
lossfn = 'categorical_crossentropy'
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [0]:
def train_model(df, name):
    X,y = prepare(df)

    models = [starting_model(10)]

    losses = []
    accuracies = []

    for model, model_name, line_style in models:
        model.compile(loss=lossfn,
              optimizer=optimizer,
              metrics=['accuracy'])

        print(model.summary())

        callbacks=[create_tensorboard_callback(name)]
        model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=callbacks)

    return model


In [0]:
def train_predict(df_train, df_test):
  X_train, y_train = prepare(df_train)
  X_test, _ = prepare(df_test, is_training=False)

  num_classes = 10
  model, _, _ = starting_model(num_classes=num_classes)

  model.compile(loss=lossfn,
        optimizer=optimizer,
        metrics=['accuracy'])
  
  model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs)

  y_pred = model.predict(X_test)
  y_pred_classes = np.argmax(y_pred, axis=1)

  submissions = pd.DataFrame({"ImageId": list(range(1,len(y_pred_classes)+1)), "Label": y_pred_classes})
  submissions.to_csv("submission.csv", index=False, header=True)


In [0]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [0]:
train_model(df_train, 'lr0.001_4lyrbnev2_bsz128_mxp')

In [27]:
train_predict(df_train, df_test)

Epoch 1/30
329/329 [==============================] - 15s 46ms/step - loss: 0.1142 - accuracy: 0.9670
Epoch 2/30
329/329 [==============================] - 15s 46ms/step - loss: 0.0440 - accuracy: 0.9869
Epoch 3/30
329/329 [==============================] - 15s 46ms/step - loss: 0.0330 - accuracy: 0.9899
Epoch 4/30
329/329 [==============================] - 15s 46ms/step - loss: 0.0267 - accuracy: 0.9921
Epoch 5/30
329/329 [==============================] - 15s 46ms/step - loss: 0.0225 - accuracy: 0.9933
Epoch 6/30
329/329 [==============================] - 15s 46ms/step - loss: 0.0193 - accuracy: 0.9939
Epoch 7/30
329/329 [==============================] - 15s 47ms/step - loss: 0.0166 - accuracy: 0.9947
Epoch 8/30
329/329 [==============================] - 15s 47ms/step - loss: 0.0145 - accuracy: 0.9955
Epoch 9/30
329/329 [==============================] - 16s 47ms/step - loss: 0.0118 - accuracy: 0.9963
Epoch 10/30
329/329 [==============================] - 16s 47ms/step - loss: 0.010

In [0]:
df_train.dtypes

label       int64
pixel0      int64
pixel1      int64
pixel2      int64
pixel3      int64
            ...  
pixel779    int64
pixel780    int64
pixel781    int64
pixel782    int64
pixel783    int64
Length: 785, dtype: object

In [0]:
import os
os.getcwd()

'/content'

In [3]:
!unzip mnist.zip

Archive:  mnist.zip
  inflating: test.csv                
  inflating: train.csv               


In [25]:
!zip -r tensorboardlogs.zip logs/

  adding: logs/ (stored 0%)
  adding: logs/fit/ (stored 0%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/ (stored 0%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/train/ (stored 0%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/train/plugins/ (stored 0%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/train/plugins/profile/ (stored 0%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/train/plugins/profile/2020_05_30_09_47_34/ (stored 0%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/train/plugins/profile/2020_05_30_09_47_34/a3e9ebb538b9.trace.json.gz (deflated 0%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/train/plugins/profile/2020_05_30_09_47_34/a3e9ebb538b9.tensorflow_stats.pb (deflated 79%)
  adding: logs/fit/20200530-094733-lr0.001_4lyrbnev2_bsz128_mxp/train/plugins/profile/2020_05_30_09_47_34/a3e9ebb538b9.kernel_stats.pb (deflated 94%)
  adding: logs/fit/2

In [0]:
!rm tensorboardlogs.zip